### Patch embedding

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/experiments/MedMNIST2D"

/content/drive/MyDrive/experiments/MedMNIST2D


In [ ]:
pip install -r requirements.txt

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/experiments/MedMNIST2D/models')

In [ ]:
# import sys
# sys.path.append(r"/home/hanwzhan/experiments/MedMNIST2D")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.utils.data as data
import PIL
import wandb

from models.patchembed import PatchEmbed23D
from models.vit import ViT

### Dataset

In [ ]:
import medmnist
from medmnist import INFO, Evaluator
from utils import create_dataloaders, get_num_classes_list


In [ ]:
data_flag_list = ['fracturemnist3d','organmnist3d','nodulemnist3d','adrenalmnist3d','dermamnist','pneumoniamnist','bloodmnist']
#,'dermamnist','pneumoniamnist','bloodmnist'
#'fracturemnist3d','organmnist3d','nodulemnist3d','adrenalmnist3d'

In [ ]:
batch_size = 8
download = True
as_rgb = True
train_datasets_list, val_datasets_list, test_datasets_list,train_dataloader_list, val_dataloader_list, test_dataloader_list = create_dataloaders(data_flag_list, batch_size, download)

Using downloaded and verified file: /root/.medmnist/fracturemnist3d.npz
Using downloaded and verified file: /root/.medmnist/fracturemnist3d.npz
Using downloaded and verified file: /root/.medmnist/fracturemnist3d.npz
Using downloaded and verified file: /root/.medmnist/organmnist3d.npz
Using downloaded and verified file: /root/.medmnist/organmnist3d.npz
Using downloaded and verified file: /root/.medmnist/organmnist3d.npz
Using downloaded and verified file: /root/.medmnist/nodulemnist3d.npz
Using downloaded and verified file: /root/.medmnist/nodulemnist3d.npz
Using downloaded and verified file: /root/.medmnist/nodulemnist3d.npz
Using downloaded and verified file: /root/.medmnist/adrenalmnist3d.npz
Using downloaded and verified file: /root/.medmnist/adrenalmnist3d.npz
Using downloaded and verified file: /root/.medmnist/adrenalmnist3d.npz
Using downloaded and verified file: /root/.medmnist/dermamnist.npz
Using downloaded and verified file: /root/.medmnist/dermamnist.npz
Using downloaded and

### Train

In [ ]:
import wandb
wandb.init(project='vit_project',name="23d_base")

# config
config = wandb.config
config.learning_rate = 0.001
config.epochs = 10
config.batch_size = 32
config.patch_size = 16
config.embed_dim = 384
config.num_classes_list = get_num_classes_list(data_flag_list)


In [ ]:

import timm
pretrained_models = timm.list_models(pretrained=True)
vit_models = [model for model in pretrained_models if 'vit' in model]

print("ViT models:", vit_models)


ViT models: ['convit_base.fb_in1k', 'convit_small.fb_in1k', 'convit_tiny.fb_in1k', 'crossvit_9_240.in1k', 'crossvit_9_dagger_240.in1k', 'crossvit_15_240.in1k', 'crossvit_15_dagger_240.in1k', 'crossvit_15_dagger_408.in1k', 'crossvit_18_240.in1k', 'crossvit_18_dagger_240.in1k', 'crossvit_18_dagger_408.in1k', 'crossvit_base_240.in1k', 'crossvit_small_240.in1k', 'crossvit_tiny_240.in1k', 'davit_base.msft_in1k', 'davit_small.msft_in1k', 'davit_tiny.msft_in1k', 'efficientvit_b0.r224_in1k', 'efficientvit_b1.r224_in1k', 'efficientvit_b1.r256_in1k', 'efficientvit_b1.r288_in1k', 'efficientvit_b2.r224_in1k', 'efficientvit_b2.r256_in1k', 'efficientvit_b2.r288_in1k', 'efficientvit_b3.r224_in1k', 'efficientvit_b3.r256_in1k', 'efficientvit_b3.r288_in1k', 'efficientvit_l1.r224_in1k', 'efficientvit_l2.r224_in1k', 'efficientvit_l2.r256_in1k', 'efficientvit_l2.r288_in1k', 'efficientvit_l2.r384_in1k', 'efficientvit_l3.r224_in1k', 'efficientvit_l3.r256_in1k', 'efficientvit_l3.r320_in1k', 'efficientvit_l3.r

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViT(
    patch_size=wandb.config.patch_size,
    num_classes_list=wandb.config.num_classes_list,
    embed_dim=wandb.config.embed_dim
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
criterion = nn.CrossEntropyLoss()


[autoreload of models.vit failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/content/drive/MyDrive/experiments/MedMNIST2D/models/vit.py", line 105
    self

RuntimeError: Invalid pretrained tag (augreg_in21) for vit_small_patch16_224.

In [ ]:
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np

NUM_EPOCHS = wandb.config.epochs
best_accuracy = 0.0

for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0.0
    total_correct_predictions = 0
    total_samples = 0
    all_labels_global = []
    all_predictions_global = []

    task_metrics = []

    for task_ids, dataset in enumerate(tqdm(train_dataloader_list, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")):
        task_loss = 0.0
        all_labels = []
        all_predictions = []
        task_samples = 0

        for inputs, labels in dataset:
            inputs = inputs.to(device)
            labels = labels.long().to(device)
            labels = labels.squeeze()
            optimizer.zero_grad()
            logits = model(inputs, task_ids)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            task_loss += loss.item() * inputs.size(0)
            task_samples += inputs.size(0)
            _, predicted_labels = torch.max(logits, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted_labels.cpu().numpy())

            all_labels_global.extend(labels.cpu().numpy())
            all_predictions_global.extend(predicted_labels.cpu().numpy())
            total_samples += inputs.size(0)
            total_loss += loss.item() * inputs.size(0)


        task_epoch_loss = task_loss / task_samples
        task_accuracy = accuracy_score(all_labels, all_predictions)
        task_f1 = f1_score(all_labels, all_predictions, average='macro', zero_division=0)

        task_metrics={
            'task_id': task_ids,
            'loss': task_epoch_loss,
            'accuracy': task_accuracy,
            'f1_score': task_f1
        }

        print(f"Task {task_metrics['task_id']} Metrics: Loss: {task_metrics['loss']:.4f}, Accuracy: {task_metrics['accuracy']:.4f}, F1 Score: {task_metrics['f1_score']:.4f}")
        wandb.log({
            f"Task {task_metrics['task_id']} Epoch Loss": task_metrics['loss'],
            f"Task {task_metrics['task_id']} Epoch Accuracy": task_metrics['accuracy'],
            f"Task {task_metrics['task_id']} Epoch F1 Score": task_metrics['f1_score']
        })

    global_loss = total_loss / total_samples
    global_accuracy = accuracy_score(all_labels_global, all_predictions_global)
    global_f1 = f1_score(all_labels_global, all_predictions_global, average='macro', zero_division=0)


    print(f"Global Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {global_loss:.4f}, Accuracy: {global_accuracy:.4f}")
    wandb.log({
        "Global Epoch Loss": global_loss,
        "Global Epoch Accuracy": global_accuracy,
        "Global Epoch F1 Score": global_f1
    })

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_samples = 0
    val_labels_global = []
    val_predictions_global = []

    for val_task_ids, val_dataset in enumerate(val_dataloader_list):
        for val_inputs, val_labels in val_dataset:
            val_inputs = val_inputs.to(device)
            val_labels = val_labels.long().to(device)
            #print(val_labels.shape[0])
            if val_labels.shape[0] == 1:
              val_labels = val_labels.squeeze()
              val_labels = val_labels.unsqueeze(0)
            else:
              val_labels = val_labels.squeeze()

            with torch.no_grad():
                val_logits = model(val_inputs, val_task_ids)
                val_loss += criterion(val_logits, val_labels).item() * val_inputs.size(0)

            _, val_predicted_labels = torch.max(val_logits, 1)
            val_labels_global.extend(val_labels.cpu().numpy())
            val_predictions_global.extend(val_predicted_labels.cpu().numpy())
            val_samples += val_inputs.size(0)

    val_loss /= val_samples
    val_accuracy = accuracy_score(val_labels_global, val_predictions_global)
    val_f1 = f1_score(val_labels_global, val_predictions_global, average='macro', zero_division=0)
    wandb.log({
        "Validation Loss": val_loss,
        "Validation Accuracy": val_accuracy,
        "Validation F1 Score": val_f1
    })

    # Update best model based on validation accuracy
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), '23d_base_best_model.pth')
        wandb.log({"Best Validation Accuracy": best_accuracy})
        print(f"Saved new best model with validation accuracy: {val_accuracy:.4f}")


In [ ]:
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def test_multi_task_model(model, criterion, test_dataloader_list, device):
    model.eval()
    task_metrics = []
    global_labels = []
    global_predictions = []

    with torch.no_grad():
        for task_id, dataloader in enumerate(test_dataloader_list):
            total_loss = 0.0
            total_samples = 0
            all_labels = []
            all_predictions = []

            for inputs, labels in tqdm(dataloader, desc=f"Testing Task {task_id}"):
                inputs = inputs.to(device)
                labels = labels.long().to(device)
                labels = labels.squeeze()

                logits = model(inputs, task_id)
                loss = criterion(logits, labels)

                total_loss += loss.item() * inputs.size(0)
                total_samples += inputs.size(0)
                _, predicted_labels = torch.max(logits, 1)
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted_labels.cpu().numpy())


            avg_loss = total_loss / total_samples
            accuracy = accuracy_score(all_labels, all_predictions)
            f1 = f1_score(all_labels, all_predictions, average='macro', zero_division=0)
            confusion = confusion_matrix(all_labels, all_predictions)

            task_metrics.append({
                'task_id': task_id,
                'loss': avg_loss,
                'accuracy': accuracy,
                'f1_score': f1,
                'confusion_matrix': confusion
            })


            offset = task_id * (max(all_labels) + 1)
            global_labels.extend([label + offset for label in all_labels])
            global_predictions.extend([pred + offset for pred in all_predictions])


    global_accuracy = accuracy_score(global_labels, global_predictions)
    global_f1 = f1_score(global_labels, global_predictions, average='macro', zero_division=0)
    global_confusion = confusion_matrix(global_labels, global_predictions)

    return task_metrics, {'accuracy': global_accuracy, 'f1_score': global_f1, 'confusion_matrix': global_confusion}


model.load_state_dict(torch.load('23d_base_best_model.pth'))
model.to(device)


task_results, global_results = test_multi_task_model(model, criterion, test_dataloader_list, device)


for result in task_results:
    print(f"Test Results for Task {result['task_id']}:")
    print(f"Loss: {result['loss']:.4f}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"F1 Score: {result['f1_score']:.4f}")
    wandb.log({
        f"Task {result['task_id']} Epoch Loss": result['loss'],
        f"Task {result['task_id']} Epoch Accuracy": result['accuracy'],
        f"Task {result['task_id']} Epoch F1 Score": result['f1_score']
    })

print("Global Test Results:")
print(f"Accuracy: {global_results['accuracy']:.4f}")
print(f"F1 Score: {global_results['f1_score']:.4f}")


wandb.log({
    "Global Test Accuracy": global_results['accuracy'],
    "Global Test F1 Score": global_results['f1_score']
})
